# Bibliothèques

In [39]:
import sqlite3
from sqlite3 import connect

import pandas as pd
pd.set_option('display.max_columns', 51)
pd.set_option('display.max_rows', 300)

# Connexion à la db

In [2]:
# Connexion à la db

# def connexion_db(file_name =""):
#     try:
#         conn = sqlite3.connect(f"{file_name}")
#         curs = conn.cursor()
#         print("connexion ok")
#     except:
#         print("no db connexion")
#     return conn, curs

# file_name = "130US_diabete_BDD.sqlite3"
# connexion_db(file_name)

try:
    conn = sqlite3.connect("130US_diabete_BDD.sqlite3") # création de l'objet qui nous connecte à la database
    curs = conn.cursor() # création d'un curseur qui permet de naviguer dans la db
    print("connexion ok")
except:
    print("no db connexion")

connexion ok


# Création des tables

In [ ]:
# Création des tables

##########
# Faire une fonction
##########

# conn, curs = connexion_db(file_name)

# Quand problème de table non trouvée indiquer : curs.execute('''DROP TABLE Film;''')

# conn.execute : permet de faire des actions dans la db (ex : create table)

# Explication = pourquoi pas de id interger autoincrement !!!! L'ajouter au rapport

'''
with conn:  
    conn.execute("""CREATE TABLE IF NOT EXISTS patients (
        id INTEGER PRIMARY KEY,
        race VARCHAR,
        gender VARCHAR,
        created_at CURRENT_TIMESTAMP,
        updated_at CURRENT_TIMESTAMP
    );""")
    

with conn: 
    conn.execute("""CREATE TABLE IF NOT EXISTS medecins (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        created_at CURRENT_TIMESTAMP,
        updated_at CURRENT_TIMESTAMP
    );""")
    
# Avant de créer mes foreign key, créer la colonne qui sera indiquer comme telle 

with conn:
    conn.execute("""CREATE TABLE IF NOT EXISTS admissions (
        id INTEGER PRIMARY KEY,
        admission_type_id INTEGER,
        discharge_disposition_id INTEGER,
        admission_source_id INTEGER,
        time_in_hospital INTEGER,
        num_lab_procedures INTEGER,
        num_procedures INTEGER,
        num_medications INTEGER,
        diag_1 VARCHAR,
        diag_2 VARCHAR,
        diag_3 VARCHAR,
        number_diagnoses INTEGER,
        max_glu_serum VARCHAR,
        a1c_result VARCHAR,
        change VARCHAR,
        diabetes_med VARCHAR,
        readmitted VARCHAR,
        age INTEGER,
        weight INTEGER,
        payer_code VARCHAR,
        number_outpatient INTEGER,
        number_emergency INTEGER,
        number_inpatient INTEGER,
        medecin_id INTEGER,
        patient_id INTEGER,
        created_at CURRENT_TIMESTAMP,
        updated_at CURRENT_TIMESTAMP,        
        FOREIGN KEY (medecin_id) REFERENCES medecins (id),
        FOREIGN KEY (patient_id) REFERENCES patients (id)
    );""")
'''
    
# conn.commit() : sauvegarde (commit) les changements/transactions / Par défaut, la connection n'est pas 
    # en autocommit (il faut committer explicitement les ordres sql). Sauf les ordres modifiant 
    # la structure de la base (comme par exemple création d'une table) qui font un commit automatiquement => ne faire
    # un commit que lorsqu'on ajoute de la donnée
    
# conn.close() : ferme la connexion

# Remplissage des tables

In [4]:
# skipinitialspace : Skip spaces after delimiter

data = pd.read_csv('diabetic_data_130US_hospital.csv', sep=';', skipinitialspace = True)
data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
df = data.copy()

## Patients

### Analyse des duplicate pour création primary key

In [6]:
patients = df[['patient_nbr','race', 'gender']]
patients

,patient_nbr,race,gender
0,8222157,Caucasian,Female
1,55629189,Caucasian,Female
2,86047875,AfricanAmerican,Female
3,82442376,Caucasian,Male
4,42519267,Caucasian,Male
...,...,...,...
101761,100162476,AfricanAmerican,Male
101762,74694222,AfricanAmerican,Female
101763,41088789,Caucasian,Male
101764,31693671,Caucasian,Female


In [7]:
patients.nunique()

patient_nbr    71518
race               6
gender             3
dtype: int64

In [8]:
# Nb de lignes dupliquées

patients.duplicated(subset = 'patient_nbr').sum()

30248

In [9]:
# On supprime les lignes dupliquées

patients = patients.drop_duplicates()
patients

,patient_nbr,race,gender
0,8222157,Caucasian,Female
1,55629189,Caucasian,Female
2,86047875,AfricanAmerican,Female
3,82442376,Caucasian,Male
4,42519267,Caucasian,Male
...,...,...,...
101754,183087545,Caucasian,Female
101755,188574944,Other,Female
101756,140199494,Other,Female
101758,120975314,Caucasian,Female


In [15]:
comptage = patients.sort_values(by = 'patient_nbr')
comptage

,patient_nbr,race,gender
4267,135,Caucasian,Female
5827,378,Caucasian,Female
67608,729,Caucasian,Female
17494,774,Caucasian,Female
2270,927,AfricanAmerican,Female
...,...,...,...
99863,189351095,Caucasian,Female
95282,189365864,Other,Male
93651,189445127,Caucasian,Female
101748,189481478,Caucasian,Female


In [17]:
# Nous avons encore des doublons dans la colonne 'patient_nbr' qu'il convient d'analyser plus précisément

comptage.duplicated(subset = 'patient_nbr').sum()

255

In [21]:
doublons = comptage[comptage.duplicated(subset = 'patient_nbr')]
doublons

,patient_nbr,race,gender
20977,453267,Caucasian,Male
12669,561609,Caucasian,Female
19576,1041291,Caucasian,Male
5064,1147059,Other,Male
7864,1226007,Other,Female
...,...,...,...
11283,113959818,AfricanAmerican,Female
95449,131425574,Caucasian,Male
95288,141045827,?,Female
92212,151228229,Asian,Male


In [40]:
doublons.sort_values(by='patient_nbr')

,patient_nbr,race,gender
20977,453267,Caucasian,Male
12669,561609,Caucasian,Female
19576,1041291,Caucasian,Male
5064,1147059,Other,Male
7864,1226007,Other,Female
11883,1243557,Caucasian,Female
8946,1260063,AfricanAmerican,Male
13486,1384182,Caucasian,Male
79469,1400022,Hispanic,Male
28744,1553220,AfricanAmerican,Male


In [53]:
# Filtre pour l'analyse 

filtre = patients[patients['patient_nbr'] == 1147059]
filtre

,patient_nbr,race,gender
1629,1147059,Caucasian,Male
5064,1147059,Other,Male


### Insertion des données 

In [36]:
for i in doublons : 
#     duplicated(subset = 'patient_nbr')== False:
    print(doublons[[i]])

        patient_nbr
20977        453267
12669        561609
19576       1041291
5064        1147059
7864        1226007
...             ...
11283     113959818
95449     131425574
95288     141045827
92212     151228229
101671    151756061

[255 rows x 1 columns]
                   race
20977         Caucasian
12669         Caucasian
19576         Caucasian
5064              Other
7864              Other
...                 ...
11283   AfricanAmerican
95449         Caucasian
95288                 ?
92212             Asian
101671                ?

[255 rows x 1 columns]
        gender
20977     Male
12669   Female
19576     Male
5064      Male
7864    Female
...        ...
11283   Female
95449     Male
95288   Female
92212     Male
101671  Female

[255 rows x 1 columns]


In [ ]:
for index, i in patients.iterrows():
    columns = tuple(i.values)
        
    sql = "INSERT INTO patients (id, race, gender) VALUES (?, ?, ?)"

    curs.execute(sql, columns)
    conn.commit() # sauvegarde des changements car pas d'autocommitted par défault

In [ ]:
# Pb avec ma table patients = 1 patient peut apparaître plusieurs fois => pas unique ! Donc pour l'instant, 
    # utilisation de l'id incrementer autoamtique mais je vais avoir id qui seront en réalité des doublons
    # => revoir modélisation ou infos présentes dans la table (ne garder que la race, le genre qui sont des données
    # fixes et mettre les autres mesures dans une autre table)
    # remplacer l'id autoincrement ar l'id du fichier excel directement

for index, i in test.iterrows():
    poire = tuple(i.values)
        
    sql = "INSERT INTO patients (race, gender, age, weight, payer_code, number_outpatient, number_emergency, number_inpatient) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"

    curs.execute(sql, poire)
    conn.commit() # sauvegarde des changements car pas d'autocommitted par défault

In [ ]:
# test_race = data.loc[:,['race']] # On sélectionne toutes les données de la colonne

# Pour chaque ligne du dataframe, faire une liste : 

# Création d'un df avec les colonnes voulues
# Transformation en list avec to_list
# Dans une boucle, insertion ligne par ligne

test = data[['race', 'gender', 'age', 'weight', 'payer_code', 'number_outpatient', 'number_emergency', 'number_inpatient']]
test

In [ ]:
# mettre le conn.close() -> revoir

In [ ]:
df = pd.read_sql_query("SELECT * FROM patients", conn)
df